# 价值迭代算法在迷宫游戏中的可视化实现

## 1. 引言

本文旨在通过一个直观的迷宫小游戏，复现并可视化强化学习中的核心算法之一——**价值迭代算法**。我们将详细阐述算法的原理，并展示其在求解贝尔曼最优方程、发现最优状态价值和最优策略过程中的动态变化。

## 2. 算法原理：价值迭代 (Value Iteration)

价值迭代是一种模型已知（Model-Based）的动态规划算法，用于解决有限马尔可夫决策过程（MDPs）。其核心思想是通过迭代更新状态价值函数，直到价值函数收敛到最优价值函数 $V^*(s)$，从而间接推导出最优策略 $\pi^*(s)$。

### 算法 4.1：价值迭代算法

**初始化：**
*   对于所有状态-动作对 $(s, a)$，转移概率模型 $p(s'|s,a)$ 和奖励概率模型 $p(r|s,a)$ 已知。
*   设定初始价值函数猜测 $v_0(s)$ **（通常初始化为全零矩阵）**。
*   设定折扣因子 $\gamma \in [0, 1)$。
*   设定收敛阈值 $\theta$（一个非常小的正数）。

**目标：**
寻找最优状态价值函数 $V^*(s)$ 和最优策略 $\pi^*(s)$，以求解贝尔曼最优方程。

**迭代过程：**
当 $v_k$ 尚未收敛（即 $\|v_k - v_{k-1}\| < \theta$ 不满足）时，对于第 $k$ 次迭代，执行以下步骤：

1.  **对于每个状态 $s \in S$：**
    *   **计算所有动作的 Q-值：** 对于该状态 $s$ 下的每个可能动作 $a \in A(s)$，计算其 Q-值 $q_k(s,a)$。Q-值的计算基于当前状态 $s$ 采取动作 $a$ 得到的即时奖励期望，加上折现后的下一个状态 $s'$ 的期望价值：
        $$q_k(s,a) = \sum_{r, s'} p(s', r | s, a) [r + \gamma v_k(s')]$$
        如果奖励和下一个状态是确定性的，则简化为：
        $$q_k(s,a) = R(s,a) + \gamma \sum_{s'} p(s'|s,a)v_k(s')$$
        在更简单的确定性环境中（如本迷宫示例），甚至可以进一步简化为：
        $$q_k(s,a) = R(s,a) + \gamma v_k(s')$$
    *   **找到最大动作价值：** 确定在状态 $s$ 下能获得最大 Q-值的动作 $a^*_k(s)$：
        $$a^*_k(s) = \arg \max_a q_k(s,a)$$
    *   **策略更新：** 更新当前策略 $\pi_{k+1}(a|s)$。如果 $a=a^*_k(s)$，则 $\pi_{k+1}(a|s) = 1$（表示选择该动作的概率为1）；否则 $\pi_{k+1}(a|s) = 0$。**（在本实现中，我们将选择一个具体的动作来表示该状态下的最优策略）。**
    *   **价值更新：** 更新状态 $s$ 的价值函数 $v_{k+1}(s)$，将其设置为在该状态下能获得的最大 Q-值：
        $$v_{k+1}(s) = \max_a q_k(s,a)$$

**收敛条件：**
迭代持续进行，直到连续两次迭代的状态价值函数之差的最大值（即无穷范数 $\|v_k - v_{k-1}\|$）小于预定义的收敛阈值 $\theta$。当算法收敛时，$v_k(s)$ 将近似于最优状态价值函数 $V^*(s)$，而 $\pi_{k+1}(a|s)$ 则近似于最优策略 $\pi^*(s)$。

## 3. 迷宫游戏环境设置

为了直观展示价值迭代过程，我们构建一个 4x4 的简单迷宫环境。迷宫中的每个方格代表一个状态。

### 3.1 迷宫网格布局

*   **网格大小：** 4x4 的二维矩阵。
*   **状态类型：**
    *   **普通方格：** 奖励为 0。
    *   **终点 (Goal)：** 奖励为 1，代表成功。
    *   **陷阱 (Trap)：** 奖励为 -1，代表失败。
*   **动作 (Actions)：** 每个方格可以向上下左右四个方向移动。如果尝试移动到迷宫外部，则会停留在原地。
    *   `'↑'` (上)
    *   `'↓'` (下)
    *   `'←'` (左)
    *   `'→'` (右)

### 3.2 矩阵表示约定

为了可视化和代码实现方便，我们定义了以下矩阵的初始状态和输出格式：

**1. 奖励矩阵 (Rewards Matrix)：**
表示迷宫中每个状态的即时奖励。
```
[
  [0,  0, -1, -1],  // (0,0)普通, (0,1)普通, (0,2)陷阱, (0,3)陷阱
  [0, -1, -1,  1],  // (1,0)普通, (1,1)陷阱, (1,2)陷阱, (1,3)终点
  [0, -1,  0,  0],  // (2,0)普通, (2,1)陷阱, (2,2)普通, (2,3)普通
  [-1, 0,  0, -1]   // (3,0)陷阱, (3,1)普通, (3,2)普通, (3,3)陷阱
]
```

**2. 初始价值矩阵 (Initial Values Matrix)：**
在价值迭代开始时，所有状态的价值都初始化为零。
```
[
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0]
]
```

**3. 初始策略矩阵 (Initial Policies Matrix)：**
在价值迭代开始时，每个状态的最优策略的初始猜测。通常初始化为空或随机策略。在我们的可视化中，它将随着价值迭代的进行而逐渐更新。初始时，我们将其设置为表示未知或未确定的空字符串。
```
[
  ['', '', '', ''],
  ['', '', '', ''],
  ['', '', '', ''],
  ['', '', '', '']
]
```
*注：策略矩阵中的箭头表示在对应状态下，根据当前价值函数评估出的最优动作方向。如果多个动作都能达到相同的最优价值，我们将根据预设的优先级（↑ > ↓ > ← > →）选择并显示其中一个。*

In [1]:
import numpy as np

# --- 1. 全局常量定义 ---
GRID_SIZE = 4                      # 迷宫的维度 (N x N)
REWARD_GOAL = 1                    # 终点奖励
REWARD_TRAP = -1                   # 陷阱奖励
REWARD_NORMAL = 0                  # 普通方格奖励

# 动作定义及对应的行列变化 (优先级: 上 > 下 > 左 > 右)
ACTIONS = ['↑', '↓', '←', '→']
ACTION_DELTAS = {
    '↑': (-1, 0),  # 向上移动：行-1
    '↓': (1, 0),   # 向下移动：行+1
    '←': (0, -1),  # 向左移动：列-1
    '→': (0, 1)    # 向右移动：列+1
}

GAMMA = 0.9                        # 折扣因子 (gamma)
THETA = 1e-4                       # 收敛阈值 (theta)

In [2]:
# --- 2. 环境初始化函数 ---
def initialize_environment():
    """
    初始化迷宫的奖励矩阵和初始价值矩阵。
    - 奖励矩阵根据预设值初始化。
    - 价值矩阵初始化为全零矩阵。
    - 策略矩阵初始化为空，将在迭代中更新。
    
    返回:
        rewards (np.array): 奖励矩阵
        values (np.array): 初始价值矩阵 (全零)
        policies (np.array): 初始策略矩阵 (空)
    """
    rewards = np.array([
        [0, 0, -1, -1],
        [0, -1, -1, 1],
        [0, -1, 0, 0],
        [-1, 0, 0, -1]
    ])
    
    # 初始价值矩阵应为全零矩阵
    values = np.zeros((GRID_SIZE, GRID_SIZE)).astype(float)
    
    # 初始策略矩阵，用空字符串表示未确定的策略
    policies = np.full((GRID_SIZE, GRID_SIZE), '', dtype=object)

    return rewards, values, policies

In [4]:
# --- 3. 辅助函数 ---
def is_valid_state(row, col):
    """
    检查给定状态 (row, col) 是否在迷宫范围内。
    
    参数:
        row (int): 行索引
        col (int): 列索引
        
    返回:
        bool: 如果状态有效则为True，否则为False
    """
    return 0 <= row < GRID_SIZE and 0 <= col < GRID_SIZE

def get_next_state_and_reward(current_row, current_col, action, rewards_matrix):
    """
    根据当前状态和动作，计算下一个状态的坐标以及从下一个状态获得的奖励。
    如果移动到无效位置，则认为智能体停留在原地。
    
    参数:
        current_row (int): 当前状态的行索引
        current_col (int): 当前状态的列索引
        action (str): 采取的动作 ('↑', '↓', '←', '→')
        rewards_matrix (np.array): 奖励矩阵
        
    返回:
        tuple: (next_row, next_col) 下一个状态的坐标
        float: reward 离开当前状态并到达下一个状态（或停留在原地）所获得的奖励
    """
    delta_row, delta_col = ACTION_DELTAS[action]
    next_row, next_col = current_row + delta_row, current_col + delta_col

    # 如果移动到无效位置，则停留在原地
    if not is_valid_state(next_row, next_col):
        next_row, next_col = current_row, current_col

    # 奖励通常是到达下一个状态后获得的奖励
    reward = rewards_matrix[next_row, next_col] 
    return (next_row, next_col), reward

In [9]:
# --- 4. 核心算法函数 ---
def calculate_q_value(row, col, action, rewards_matrix, values_matrix, gamma):
    """
    计算给定状态 (row, col) 和动作 (action) 的 Q 值。
    假设转移是确定性的 (p(s'|s,a)=1, p(r|s,a)=1)。
    
    参数:
        row (int): 当前状态的行索引
        col (int): 当前状态的列索引
        action (str): 采取的动作
        rewards_matrix (np.array): 奖励矩阵
        values_matrix (np.array): 当前价值矩阵 (v_k)
        gamma (float): 折扣因子
        
    返回:
        float: 计算出的 Q 值
    """
    (next_row, next_col), next_state_reward = get_next_state_and_reward(row, col, action, rewards_matrix)
    
    # Q(s,a) = R_s,a + gamma * V(s')
    # 这里 R_s,a 我们简化为到达 next_state_reward
    q_value = next_state_reward + gamma * values_matrix[next_row, next_col]
    return q_value

def value_iteration_step(rewards_matrix, current_values_matrix, gamma):
    """
    执行一次价值迭代的步骤，更新价值矩阵和策略矩阵。
    
    参数:
        rewards_matrix (np.array): 奖励矩阵
        current_values_matrix (np.array): 当前迭代的价值矩阵 (v_k)
        gamma (float): 折扣因子
        
    返回:
        new_values_matrix (np.array): 更新后的价值矩阵 (v_{k+1})
        new_policies_matrix (np.array): 更新后的策略矩阵 (pi_{k+1})
        delta (float): 本次迭代中价值函数变化的最大差值
    """
    new_values_matrix = np.copy(current_values_matrix)
    new_policies_matrix = np.full((GRID_SIZE, GRID_SIZE), '', dtype=object)
    
    delta = 0  # 用于记录本次迭代的最大价值变化

    for r in range(GRID_SIZE):
        for c in range(GRID_SIZE):

            max_q_value_for_state = -np.inf # 记录当前状态 s 的最大 Q 值
            best_actions_for_state = []     # 记录导致最大 Q 值的动作

            # 对于当前状态 s，尝试所有可能的动作 a
            for action in ACTIONS:
                q_val = calculate_q_value(r, c, action, rewards_matrix, current_values_matrix, gamma)
                
                # 更新最大 Q 值和对应的动作
                if q_val > max_q_value_for_state:
                    max_q_value_for_state = q_val
                    best_actions_for_state = [action] # 找到更大的 Q 值，清空旧的
                elif q_val == max_q_value_for_state:
                    best_actions_for_state.append(action) # Q 值相同，添加动作

            # 更新当前状态的价值 V_{k+1}(s) = max_a Q(s,a)
            new_values_matrix[r, c] = max_q_value_for_state
            
            # 修正: 策略只显示一个最佳动作。
            # 如果有多个最佳动作，按照优先级选择第一个：↑ > ↓ > ← > →
            if best_actions_for_state:
                # 按照ACTIONS列表中定义的优先级来选择第一个最佳动作
                chosen_policy_action = ''
                for preferred_action in ACTIONS:
                    if preferred_action in best_actions_for_state:
                        chosen_policy_action = preferred_action
                        break
                new_policies_matrix[r, c] = chosen_policy_action
            else:
                new_policies_matrix[r, c] = '' # 没有找到最佳动作 (不应发生，除非无动作)

            # 计算最大变化量 ||v_k - v_{k-1}||_inf
            delta = max(delta, abs(new_values_matrix[r, c] - current_values_matrix[r, c]))

    return new_values_matrix, new_policies_matrix, delta

In [6]:
# --- 5. 可视化/打印函数 ---
def print_matrix(matrix, name, decimal_places=4):
    """
    以格式化的方式打印矩阵，方便可视化。
    
    参数:
        matrix (np.array): 要打印的矩阵
        name (str): 矩阵的名称，用于输出标题
        decimal_places (int): 对于浮点数，保留的小数位数
    """
    print(f"\n--- {name} ---")
    for row in matrix:
        print("[", end="")
        for i, val in enumerate(row):
            if isinstance(val, (int, float, np.floating)):
                print(f"{val:{decimal_places + 3}.{decimal_places}f}", end="") # 格式化浮点数
            else:
                print(f"{val:8}", end="") # 格式化字符串
            if i < len(row) - 1:
                print(", ", end="")
        print("]")

In [7]:
# --- 6. 主执行函数 ---
def visualize_value_iteration():
    """
    执行并可视化整个价值迭代过程，直到收敛。
    """
    # 1. 初始化环境
    rewards, values, policies = initialize_environment()
    
    print("--------------------------------------------------")
    print("--------------- 价值迭代可视化启动 ---------------")
    print("--------------------------------------------------")
    print_matrix(rewards, "奖励矩阵 (Rewards)")
    print_matrix(values, "初始价值矩阵 (V0)")
    print_matrix(policies, "初始策略矩阵 (π0)")

    iteration = 0
    delta = float('inf') # 初始delta设置为无穷大，确保至少执行一次迭代

    # 2. 迭代直到收敛
    while delta > THETA:
        iteration += 1
        print(f"\n\n==================== 迭代 {iteration} ====================")
        
        # 记录本轮迭代前的价值矩阵，用于计算delta
        old_values = np.copy(values) 
        
        # 执行一次价值迭代步骤
        values, policies, delta = value_iteration_step(rewards, values, GAMMA)
        
        # 打印本轮迭代的结果
        print_matrix(values, f"价值矩阵 (V{iteration})")
        print_matrix(policies, f"策略矩阵 (π{iteration})")
        print(f"最大价值变化量 (Delta): {delta:.6f}")
        
    print("\n--------------------------------------------------")
    print("--------------- 价值迭代收敛完成 ---------------")
    print("--------------------------------------------------")
    print_matrix(values, "最终价值矩阵 (V*)")
    print_matrix(policies, "最终最优策略矩阵 (π*)")
    print(f"\n总计迭代次数: {iteration}")

In [10]:
visualize_value_iteration()

--------------------------------------------------
--------------- 价值迭代可视化启动 ---------------
--------------------------------------------------

--- 奖励矩阵 (Rewards) ---
[       0,        0,       -1,       -1]
[       0,       -1,       -1,        1]
[       0,       -1,        0,        0]
[      -1,        0,        0,       -1]

--- 初始价值矩阵 (V0) ---
[ 0.0000,  0.0000,  0.0000,  0.0000]
[ 0.0000,  0.0000,  0.0000,  0.0000]
[ 0.0000,  0.0000,  0.0000,  0.0000]
[ 0.0000,  0.0000,  0.0000,  0.0000]

--- 初始策略矩阵 (π0) ---
[        ,         ,         ,         ]
[        ,         ,         ,         ]
[        ,         ,         ,         ]
[        ,         ,         ,         ]


==================== 迭代 1 ====================

--- 价值矩阵 (V1) ---
[ 0.0000,  0.0000,  0.0000,  1.0000]
[ 0.0000,  0.0000,  1.0000,  1.0000]
[ 0.0000,  0.0000,  0.0000,  1.0000]
[ 0.0000,  0.0000,  0.0000,  0.0000]

--- 策略矩阵 (π1) ---
[↑       , ↑       , ←       , ↓       ]
[↑       , ↑       , →       , →      